In [ ]:
# Tensorflow / Keras
import tensorflow as tf
from tensorflow import keras
print('Tensorflow version:', tf.__version__)
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras.layers import Input # for instantiating a keras tensor
from keras.layers import GRU, Dense, GRU, Dropout, concatenate, Activation, Concatenate, Flatten, BatchNormalization# for creating layers inside the Neural Network
from keras.models import load_model
from keras.optimizers import Adam, Nadam
from keras.initializers import he_normal
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit

# Visualization
import plotly
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

import scipy.io
from scipy.interpolate import interp1d
print('scipy: %s' % scipy.__version__) # print version

import matplotlib.pyplot as plt
import math
import os.path

<h1>Data posprocessing</h1>
<h3>Concatinating data from several csv files to one</h3>
This section synchronizes and concatinates the EIM and kinematic data through unix time.

In [ ]:
# Sliding window averaging helper function. Check report for details
def rolling_mean(input_signal):
    output_signal = input_signal.copy()
    buffer = len(input_signal) // 50
    running_sum = 0.0

    for i in range(len(input_signal)):
        running_sum += input_signal[i]

        if i < buffer:
            output_signal[i] = running_sum / float(i + 1)
        else:
            running_sum -= input_signal[i - buffer]
            output_signal[i] = running_sum / float(buffer)

    return output_signal

In [ ]:
# Set Pandas options to display more columns
pd.options.display.max_columns=150

# Loading data and adding headers 'ElbowAngles' and 'Time'
# NB! The concatination is for now a manual process, where the sample directory for
# elbow angles and EIM data must be specified for each sample pr run of this cell!
kinematic_data_dir = 'MoCap/test_on_simon/24_45deg_hold_6kg/elbow_angles.csv'
eim_data_dir = 'MoCap/test_on_simon/24_45deg_hold_6kg/filtered_output_data.csv'
df_kin=pd.read_csv(kinematic_data_dir, encoding='utf-8', names=["ElbowAngles", "Time"])
df_eim=pd.read_csv(eim_data_dir, encoding='utf-8')

# Stripping data of unwanted delimiters and converting to float
df_kin['ElbowAngles'] = df_kin['ElbowAngles'].str.strip('[]').astype(float)
df_kin['Time'] = df_kin['Time'].str.strip('[]').astype(str)
df_kin['Time'] = df_kin['Time'].str.strip('\'').astype(float)

# Extracting Unix time
kin_unix = df_kin['Time'].values
eim_unix = df_eim['Time'].values

# Extracting max and min Unix values of kin and using these to figure out where to slice the EIM data
kin_min = kin_unix.min()
kin_max = kin_unix.max()
eim_min = eim_unix.min()
eim_max = eim_unix.max()

# Filter EIM data based on kinematic Unix time range
df_eim = df_eim[(df_eim['Time'] >= kin_min) & (df_eim['Time'] <= kin_max-1.5)]

# Creating timestamps. EIM samples at 1000hZ, so 1 timestamp will correspond to 1ms
df_eim['Timestamp'] = np.linspace(0, (len(df_eim) - 1), len(df_eim))
df_kin['Timestamp'] = np.linspace(0, (len(df_kin) - 1), len(df_kin))

# Extract timestamps and kinematic data
kinematic_timestamps = df_kin['Timestamp'].values
kinematic_data = df_kin['ElbowAngles'].values
eim_timestamps = df_eim['Timestamp'].values

# Interpolation of the kinematic data to match the EIM data
shape = eim_timestamps.shape
kinematic_interpolated = np.empty(shape)
kin_idx = 0
step = math.floor(len(eim_timestamps)/len(kinematic_timestamps))
step_remainder = len(eim_timestamps)/len(kinematic_timestamps) - step
step_temp = 0
temp = 0

# Linear interpolation done by averaging between two points. Each data step is done based 
# on the integer difference between the lenghts of the datasets. For increased accuracy, 
# whenever the remainder of the division becomes equal to or greater than 1, 1 is added 
# to the step, and withdrawn from the counter.
for i in range(0, len(kinematic_data), 1):
    kinematic_interpolated[kin_idx] = kinematic_data[i]
    if i < len(kinematic_data)-1:
        temp = kinematic_data[i+1]
        for j in range(kin_idx + 1, kin_idx+step, 1):
            kinematic_interpolated[j] = (kinematic_interpolated[j-1] + temp) / 2
    else:
        temp = kinematic_data[i]
        for j in range(kin_idx + 1, len(eim_timestamps), 1):
            kinematic_interpolated[j] = (kinematic_interpolated[j-1] + temp) / 2

        step_temp = step_temp+step_remainder
    if step_temp >= 1:
        kin_idx = kin_idx + step + 1
        step_temp = step_temp - 1
    else:
        kin_idx = kin_idx + step


# Interpolate kinematic data to match EIM timestamps. Lines are drawn between the spread out data. 
# Missing values are being extrapolated
kinematic_interpolated = interp1d(eim_timestamps, kinematic_interpolated, kind='linear', fill_value='extrapolate')

# The interpolated data is being saved to the JointAngle column in the eim_data
df_eim['JointAngle'] = kinematic_interpolated(eim_timestamps)


# Calculating rolling mean
df_eim['RollingAverageMag'] = rolling_mean(df_eim['EIMMagnitude'])
df_eim['RollingAveragePhase'] = rolling_mean(df_eim['EIMPhase'])
df_eim['RollingStdEIM'] = df_eim['EIMMagnitude'].rolling(100).std()
std_value=df_eim['RollingStdEIM'][99]
df_eim['RollingStdEIM'].fillna(value=std_value, inplace=True) 



# Saving the result to csv, where all samples are gonna be saved. Probs the user for sample number and weight
# If file does not exist it will create one.
# NB! One sample is saved in the file pr run of this cell! Check the visualization for sanity check after each run.
# If sample needs further processing or if you accidentally saves unwanted data in the main csv file, you'll have
# to manually remove that sample from the main csv.
df_final = df_eim[['Sample', 'EIMMagnitude', 'EIMPhase', 'JointAngle', 'Mass', 'Time', 'RollingAverageMag', 'RollingAveragePhase']]
df_eim_kin = 'all_samples.csv'

if(os.path.isfile(df_eim_kin)):
    sample_number = input("Please provide the sample number and press enter:")
    mass = input("Please provide the mass used in the current sample:")
    df_final = df_final.assign(Sample=sample_number)
    df_final = df_final.assign(Mass=mass)
    df_final.to_csv(df_eim_kin, mode='a', index= False, header=False)
else:
    sample_number = input("Please provide the sample number and press enter:")
    mass = input("Please provide the mass used in the current sample:")
    df_final = df_final.assign(Sample=sample_number)
    df_final = df_final.assign(Mass=mass)
    df_final.to_csv(df_eim_kin, mode='w', index= False)




# Assuming df_eim is your DataFrame
plt.plot(df_eim['Timestamp'], df_eim['JointAngle'], marker='o')
plt.xlabel('Timestamp')
plt.ylabel('ElbowAngles')
plt.title('Line Plot of ElbowAngles')
plt.show()

# plt.plot(df_eim['Timestamp'], df_eim['EIMMagnitude'], marker='o')
# plt.xlabel('Timestamp')
# plt.ylabel('EIMMagnitude')
# plt.title('Line Plot of EIMMagnitude')
# plt.show()

plt.plot(df_eim['Timestamp'], df_eim['RollingAverageMag'], marker='o')
plt.xlabel('Timestamp')
plt.ylabel('RollingAverageEIM')
plt.title('Line Plot of RollingAverageEIM')
plt.show()

plt.plot(df_eim['Timestamp'], df_eim['RollingStdEIM'], marker='o')
plt.xlabel('Timestamp')
plt.ylabel('RollingStdEIM')
plt.title('Line Plot of RollingStdEIM')
plt.show()

plt.plot(df_eim['Timestamp'], df_eim['RollingAveragePhase'], marker='o')
plt.xlabel('Timestamp')
plt.ylabel('EIMPhase')
plt.title('Line Plot of EIMPhase')
plt.show()

<h1>Load data</h1>
Once you have concatinated all samples into one csv file, you are good to load that file and start the proces.

In [ ]:
dir_all = 'all_samples.csv'
df_all=pd.read_csv(dir_all, encoding='utf-8')
grouped = df_all.groupby('Sample')

<h1>Feature extraction</h1>
Adjust as needed, whatever features you think might be interesting based on your domain knowledge.

In [ ]:
# Set Pandas options to display more columns
pd.options.display.max_columns=150

# Calculate the median for each group
median_values = grouped[['EIMMagnitude', 'EIMPhase']].apply(lambda group: group[['EIMMagnitude', 'EIMPhase']].agg(['min', 'max']).median())

# Calculate the mean for each group
mean_values = grouped[['EIMMagnitude', 'EIMPhase']].mean()

# Calculate the standard deviation for each group
standard_deviations = grouped[['EIMMagnitude', 'EIMPhase']].std()

# Calculate the variance for each group
variance_values = grouped[['EIMMagnitude', 'EIMPhase']].var()

# Calculate the kurtosis for each group
kurtosis_values = grouped[['EIMMagnitude', 'EIMPhase']].apply(pd.DataFrame.kurtosis)

# Reset the index to get the 'Sample' column back
median_values.reset_index(inplace=True)
mean_values.reset_index(inplace=True)
standard_deviations.reset_index(inplace=True)
variance_values.reset_index(inplace=True)
kurtosis_values.reset_index(inplace=True)

# Rename the columns to indicate they represent the median
median_values.columns = ['Sample', 'MedianEIMMagnitude', 'MedianEIMPhase']
mean_values.columns = ['Sample', 'MeanEIMMagnitude', 'MeanEIMPhase']
standard_deviations.columns = ['Sample', 'StdEIMMagnitude', 'StdEIMPhase']
variance_values.columns = ['Sample', 'VarEIMMagnitude', 'VarEIMPhase']
kurtosis_values.columns = ['Sample', 'KurtEIMMagnitude', 'KurtEIMPhase']

# Merge the median and mean values back into the original DataFrame based on the 'Sample' column
df_all = df_all.merge(median_values, on='Sample', how='left')
df_all = df_all.merge(mean_values, on='Sample', how='left')
df_all = df_all.merge(standard_deviations, on='Sample', how='left')
df_all = df_all.merge(variance_values, on='Sample', how='left')
df_all = df_all.merge(kurtosis_values, on='Sample', how='left')

# Calculate rate of change for each group
df_all['ROCEIMMagnitude'] = df_all['RollingAverageMag'].pct_change()
df_all['ROCEIMPhase'] = df_all['RollingAveragePhase'].pct_change()

#Filling NaN values out with the first mean value in the series
ROC_value=df_all['ROCEIMMagnitude'][1]
df_all['ROCEIMMagnitude'].fillna(value=ROC_value, inplace=True)
ROC_value=df_all['ROCEIMPhase'][1]
df_all['ROCEIMPhase'].fillna(value=ROC_value, inplace=True)

In [ ]:
df_all

In [ ]:

# Assuming your data is stored in a pandas DataFrame named 'df'
# Filter the data for the last 10 samples
df_last_10_samples = df_all[df_all['Sample'] >= 89]

# Create a scatter plot for EIMMagnitude and JointAngle
fig = go.Figure()

for sample in df_last_10_samples['Sample'].unique():
    sample_data = df_last_10_samples[df_last_10_samples['Sample'] == sample]
    fig.add_trace(go.Scatter(x=sample_data.index, y=sample_data['EIMMagnitude'],
                             mode='lines+markers', name=f'Sample {sample} - EIMMagnitude'))
    fig.add_trace(go.Scatter(x=sample_data.index, y=sample_data['JointAngle'],
                             mode='lines+markers', name=f'Sample {sample} - JointAngle'))

fig.update_layout(title='EMG and Kinematic Data for the last 10 Samples',
                  xaxis_title='Time Steps', yaxis_title='Value')

# Show the plot
fig.show()

<h1>Create test set</h1>
These will be extracted from dataframe after normalization.

In [ ]:
df_all_grouped = df_all.groupby(['Sample'])

specific_samples = [23, 38, 47, 50, 89, 98]

# Create an empty DataFrame to store the selected samples
df_test = pd.DataFrame()

# Iterate through the specific samples and extract them for testing.
for sample_value in specific_samples:
    if sample_value in df_all_grouped.groups:
        df_test = pd.concat([df_test, df_all_grouped.get_group(sample_value)])

# Save the test data to csv
# df_test.to_csv('/content/drive/MyDrive/NeuralNetwork/test_samples.csv')

In [ ]:
df_test

In [ ]:
# Define the columns you want to normalize
features_to_normalize = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                        'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                        'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                        'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                        'ROCEIMPhase']

targets_to_normalize = ['Mass', 'JointAngle']

columns_to_use = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                        'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                        'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                        'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                        'ROCEIMPhase', 'Mass', 'JointAngle']

# Extract the 'Sample' column to append after normalization
sample_column = df_all['Sample'].values

# Create a copy of an original dataframe
df2=df_all.drop(['Time', 'Sample'], axis=1)

# Extracting mean and standard deviation for mean normalization
df_mean = df2.mean()
df_std = df2.std()
normalized_df=(df2-df_mean)/df_std

# Add the sample column again
normalized_df['Sample'] = sample_column

# Save means and std to CSV
# df_mean.to_csv('/content/drive/MyDrive/NeuralNetwork/means.csv', header=True)
# df_std.to_csv('/content/drive/MyDrive/NeuralNetwork/std_devs.csv', header=True)

# Show a snaphsot of data
normalized_df

In [ ]:
df_org = normalized_df*df_std+df_mean
df_org

In [ ]:
# Display the original DataFrame length
print("Original DataFrame Length:", len(normalized_df))

# Remove the extracted samples from the original DataFrame
normalized_df.drop(normalized_df[normalized_df['Sample'] == 23].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 38].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 47].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 50].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 89].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 98].index, inplace = True)

# Display the modified original DataFrame length
print("\nModified Original DataFrame Length:", len(normalized_df))

In [ ]:
# Check if test samples have been removed
for sample_value in specific_samples:
  print(sample_value in normalized_df['Sample'].unique())

In [ ]:
normalized_df

<h1>Create the sequences</h1>

In [ ]:
# Set number of features and sequence lengths
num_features = 16
sequence_length = 30

columns_x = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
            'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
            'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
            'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
            'ROCEIMPhase']

columns_y = ['JointAngle', 'Mass']

# Group the data by the 'Sample' column
df_grouped = normalized_df.groupby(['Sample'])

# Create sequences for each group
X_seq, y_seq = [], []

for group_name, group_data in df_grouped:

    group_data_x_temp = group_data[columns_x]
    group_data_x = np.array(group_data_x_temp)  # Convert to NumPy array

    group_data_temp_y = group_data[columns_y]
    group_data_y = np.array(group_data_temp_y)  # Convert to NumPy array

    # Create sequences
    for i in range(len(group_data) - sequence_length + 1):
        X_seq.append(group_data_x[i:i+sequence_length, :])
        y_seq.append(group_data_y[i+sequence_length-1, :])

# Convert to numpy arrays
X_seq_np = np.array(X_seq)
y_seq_np = np.array(y_seq)

# Split into training and validation data in a 80/20 ratio. Testing is done with the samples extracted from the dataset earlier
X_train, X_val, y_train, y_val = train_test_split(X_seq_np, y_seq_np, test_size=0.2, random_state=42)

<h1>Define the network</h1>

In [ ]:
lr = 0.0002957
fc_layer_size1 = 256
fc_layer_size2 = 64
dropout_val = 0.5
recurrent_dropout_val = 0.4

# Set random seed for reproducibility
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Build the GRU model
model = Sequential()

model.add(GRU(units=fc_layer_size1, activation='relu', return_sequences=True,
              kernel_initializer=he_normal(seed=seed_value),
              kernel_regularizer=l2(0.01),
              input_shape=(sequence_length, num_features), name='Input-Layer'))
model.add(BatchNormalization())
model.add(Dropout(dropout_val, seed=seed_value))  # Dropout after the first GRU layer

model.add(GRU(units=fc_layer_size2, activation='relu',
              kernel_initializer=he_normal(seed=seed_value),
              kernel_regularizer=l2(0.01),
              recurrent_dropout=recurrent_dropout_val, dropout=recurrent_dropout_val))
model.add(BatchNormalization())
model.add(Dropout(dropout_val, seed=seed_value))  # Dropout after the second GRU layer

model.add(Dense(units=2, activation='linear'))  # Output layer

optimizer = Nadam(learning_rate = lr, beta_1 = 0.9, beta_2 = 0.999, clipvalue = 0.5, clipnorm = 1)  # Clip gradients between -0.5 and 0.5 to address the exploding gradient problem
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

<h1>Train the model</h1>

In [ ]:
batch = 72
num_epochs = 200

# Train the model and record the history. Change model_checkpoint name as needed
checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Define the EarlyStopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',  # Choose the metric to monitor
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored metric
)

history = model.fit(X_train,
                    y_train,
                    epochs=num_epochs,
                    batch_size=72,
                    verbose=1,
                    callbacks=[model_checkpoint_callback, early_stopping_callback],
                    validation_data=(X_val, y_val),
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    validation_freq=1,
                   )

# Save the model. Adjust name accordingly
model_dir = 'final_model3_Nadam_lz256x64_ES_Drop05.h5'
model.save(model_dir)
print("Model saved successfully.")

In [ ]:
# Plot training loss, validation loss, training accuracy, and validation accuracy
plt.figure(figsize=(12, 6))

# Plot Training Loss and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Training Accuracy and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

<h1>Hyper parameters tuning</h1>

# Introduction to Hyperparameter Sweeps

Searching through high dimensional hyperparameter spaces to find the most performant model can get unwieldy very fast. Hyperparameter sweeps provide an organized and efficient way to conduct a battle royale of models and pick the most accurate model. They enable this by automatically searching through combinations of hyperparameter values (e.g. learning rate, batch size, number of hidden layers, optimizer type) to find the most optimal values.

In this tutorial we'll see how you can run sophisticated hyperparameter sweeps in 3 easy steps using Weights and Biases.

![](https://i.imgur.com/WVKkMWw.png)

## Sweeps: An Overview

Running a hyperparameter sweep with Weights & Biases is very easy. There are just 3 simple steps:

1. **Define the sweep:** we do this by creating a dictionary or a [YAML file](https://docs.wandb.com/library/sweeps/configuration) that specifies the parameters to search through, the search strategy, the optimization metric et all.

2. **Initialize the sweep:** with one line of code we initialize the sweep and pass in the dictionary of sweep configurations:
`sweep_id = wandb.sweep(sweep_config)`

3. **Run the sweep agent:** also accomplished with one line of code, we call wandb.agent() and pass the sweep_id to run, along with a function that defines your model architecture and trains it:
`wandb.agent(sweep_id, function=train)`

And voila! That's all there is to running a hyperparameter sweep! In the notebook below, we'll walk through these 3 steps in more detail.


We highly encourage you to fork this notebook, tweak the parameters, or try the model with your own dataset!

## Resources
- [Sweeps docs →](https://docs.wandb.com/library/sweeps)
- [Launching from the command line →](https://www.wandb.com/articles/hyperparameter-tuning-as-easy-as-1-2-3)


# Setup
Start out by installing the experiment tracking library and setting up your free W&B account:


*   **pip install wandb** – Install the W&B library
*   **import wandb** – Import the wandb library


In [ ]:
# WandB – Install the W&B library
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

In [ ]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, GRU, BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop, SGD, Adam, Nadam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping
from keras.initializers import he_normal
from keras.regularizers import l2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print('Tensorflow version:', tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.model_selection import train_test_split

import wandb
from wandb.keras import WandbCallback

In [ ]:
# For Google Colab, uncomment following:

# from google.colab import drive
# import joblib
# drive.mount('/content/drive')

In [ ]:
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

dir_all = '/content/drive/MyDrive/NeuralNetwork/all_samples.csv'
df_all=pd.read_csv(dir_all, encoding='utf-8')
grouped = df_all.groupby('Sample')

In [ ]:
# Set Pandas options to display more columns
pd.options.display.max_columns=150

# Calculate the median for each group
median_values = grouped[['EIMMagnitude', 'EIMPhase']].apply(lambda group: group[['EIMMagnitude', 'EIMPhase']].agg(['min', 'max']).median())

# Calculate the mean for each group
mean_values = grouped[['EIMMagnitude', 'EIMPhase']].mean()

# Calculate the standard deviation for each group
standard_deviations = grouped[['EIMMagnitude', 'EIMPhase']].std()

# Calculate the variance for each group
variance_values = grouped[['EIMMagnitude', 'EIMPhase']].var()

# Calculate the kurtosis for each group
kurtosis_values = grouped[['EIMMagnitude', 'EIMPhase']].apply(pd.DataFrame.kurtosis)

# Reset the index to get the 'Sample' column back
median_values.reset_index(inplace=True)
mean_values.reset_index(inplace=True)
standard_deviations.reset_index(inplace=True)
variance_values.reset_index(inplace=True)
kurtosis_values.reset_index(inplace=True)

# Rename the columns to indicate they represent the median
median_values.columns = ['Sample', 'MedianEIMMagnitude', 'MedianEIMPhase']
mean_values.columns = ['Sample', 'MeanEIMMagnitude', 'MeanEIMPhase']
standard_deviations.columns = ['Sample', 'StdEIMMagnitude', 'StdEIMPhase']
variance_values.columns = ['Sample', 'VarEIMMagnitude', 'VarEIMPhase']
kurtosis_values.columns = ['Sample', 'KurtEIMMagnitude', 'KurtEIMPhase']

# Merge the median and mean values back into the original DataFrame based on the 'Sample' column
df_all = df_all.merge(median_values, on='Sample', how='left')
df_all = df_all.merge(mean_values, on='Sample', how='left')
df_all = df_all.merge(standard_deviations, on='Sample', how='left')
df_all = df_all.merge(variance_values, on='Sample', how='left')
df_all = df_all.merge(kurtosis_values, on='Sample', how='left')

# Calculate rate of change for each group
df_all['ROCEIMMagnitude'] = df_all['RollingAverageMag'].pct_change()
df_all['ROCEIMPhase'] = df_all['RollingAveragePhase'].pct_change()

#Filling NaN values out with the first mean value in the series
ROC_value=df_all['ROCEIMMagnitude'][1]
df_all['ROCEIMMagnitude'].fillna(value=ROC_value, inplace=True)
ROC_value=df_all['ROCEIMPhase'][1]
df_all['ROCEIMPhase'].fillna(value=ROC_value, inplace=True)

In [ ]:
df_all

In [ ]:
# Define the columns you want to normalize
features_to_normalize = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                        'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                        'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                        'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                        'ROCEIMPhase']

targets_to_normalize = ['Mass', 'JointAngle']

columns_to_use = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                        'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                        'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                        'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                        'ROCEIMPhase', 'Mass', 'JointAngle']

# Extract the 'Sample' column to append after normalization
sample_column = df_all['Sample'].values

# Create a copy of an original dataframe
df2=df_all.drop(['Time', 'Sample'], axis=1)

# Extracting mean and standard deviation for mean normalization
df_mean = df2.mean()
df_std = df2.std()
normalized_df=(df2-df_mean)/df_std

# Add the sample column again
normalized_df['Sample'] = sample_column

# Save means and std to CSV
# df_mean.to_csv('/content/drive/MyDrive/NeuralNetwork/means.csv', header=True)
# df_std.to_csv('/content/drive/MyDrive/NeuralNetwork/std_devs.csv', header=True)

# Show a snaphsot of data
normalized_df

In [ ]:
# Display the original DataFrame length
print("Original DataFrame Length:", len(normalized_df))

# Remove the extracted samples from the original DataFrame
normalized_df.drop(normalized_df[normalized_df['Sample'] == 23].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 38].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 47].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 50].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 89].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 98].index, inplace = True)

# Display the modified original DataFrame length
print("\nModified Original DataFrame Length:", len(normalized_df))

In [ ]:
# Check if test samples have been removed
specific_samples = [23, 38, 47, 50, 89, 98]

for sample_value in specific_samples:
  print(sample_value in normalized_df['Sample'].unique())

## 1. Define the Sweep

Weights & Biases sweeps give you powerful levers to configure your sweeps exactly how you want them, with just a few lines of code. The sweeps config can be defined as a dictionary or a [YAML file](https://docs.wandb.com/library/sweeps).

Let's walk through some of them together:
*   **Metric** – This is the metric the sweeps are attempting to optimize. Metrics can take a `name` (this metric should be logged by your training script) and a `goal` (maximize or minimize).
*   **Search Strategy** – Specified using the 'method' variable. We support several different search strategies with sweeps.
  *   **Grid Search** – Iterates over every combination of hyperparameter values.
  *   **Random Search** – Iterates over randomly chosen combinations of hyperparameter values.
  *   **Bayesian Search** – Creates a probabilistic model that maps hyperparameters to probability of a metric score, and chooses parameters with high probability of improving the metric. The objective of Bayesian optimization is to spend more time in picking the hyperparameter values, but in doing so trying out fewer hyperparameter values.
*   **Stopping Criteria** – The strategy for determining when to kill off poorly peforming runs, and try more combinations faster. We offer several custom scheduling algorithms like [HyperBand](https://arxiv.org/pdf/1603.06560.pdf) and Envelope.
*   **Parameters** – A dictionary containing the hyperparameter names, and discreet values, max and min values or distributions from which to pull their values to sweep over.

You can find a list of all configuration options [here](https://docs.wandb.com/library/sweeps/configuration).

In [ ]:
# Setting the search mode. Choices: grid, random, bayes.
# Use random over grid if you have many hyperparemeters to tune, since grid is computational heavy
sweep_config = {
    'method': 'random'
    }

In [ ]:
# To tell the sweep which metric to optimize and in which direction. Only necesary for bayes, but a good idea regardless
metric = {
    'name': 'accuracy',
    'goal': 'maximize'
    }

sweep_config['metric'] = metric

In [ ]:
# Specifying hyperparameters we need to optimize
parameters_dict = {
    # 'optimizer': {
    #     'values': ['adam', 'nadam', 'sgd', 'rmsprop']
    #     },
    # 'learning_rate': {
    #     # a flat distribution between 0 and 0.1
    #     'distribution': 'uniform',
    #     'min': 0,
    #     'max': 0.1
    #     },
    # 'batch_size': {
    #     # integers between 32 and 256
    #     # with evenly-distributed logarithms
    #     'distribution': 'q_log_uniform_values',
    #     'q': 8,
    #     'min': 32,
    #     'max': 256,
    #     },
    # 'fc_layer_size1': {
    #     'values': [32, 64, 128, 256]
    #     },
    # 'fc_layer_size2': {
    #     'values': [32, 64, 128, 256]
    #     },
    # 'dropout': {
    #     'values': [0.2, 0.3, 0.4, 0.5]
    #     },
    'beta_1' : {
        'distribution': 'uniform',
        'min': 0.85,
        'max': 0.95
        },
    'beta_1' : {
        'distribution': 'uniform',
        'min': 0.999,
        'max': 0.9999
        },
    'epsilon' : {
        'distribution': 'uniform',
        'min': 1e-9,
        'max': 1e-7
        },
    'clipnorm' : {
        'distribution': 'uniform',
        'min': 0.1,
        'max': 10
        }
    # 'reccurent_dropout': {
    #     'values': [0.2, 0.3, 0.4, 0.5]
    #     },
    # 'kernel_regularizer': {
    #     # a flat distribution between 0 and 0.1
    #     'distribution': 'uniform',
    #     'min': 0.01,
    #     'max': 0.4
    # }


    # 'fc_layer_size': {
    #     'values': [128, 256, 512]
    #     },
    # 'dropout': {
    #       'values': [0.3, 0.4, 0.5]
    #     },
    # 'activation': {
    #     'values': ['relu', 'elu', 'selu', 'softmax']
    #     },
    # 'weight_decay': {
    #     'values': [0.0005, 0.005, 0.05]
    #     }
    }

sweep_config['parameters'] = parameters_dict

In [ ]:
# For setting values that we don't want to vary in the script.
parameters_dict.update({
    'epochs': {
        'value': 10}
    })

parameters_dict.update({
    'sequence_length': {
        'value': 30}
    })

parameters_dict.update({
    'batch_size': {
        'value': 72}
    })

parameters_dict.update({
    'learning_rate': {
        'value': 0.0002957}
    })

parameters_dict.update({
    'optimizer': {
        'value': 'nadam'}
    })

parameters_dict.update({
    'fc_layer_size1': {
        'value': 256}
    })

parameters_dict.update({
    'fc_layer_size2': {
        'value': 64}
    })

parameters_dict.update({
    'dropout': {
        'value': 0.4}
    })

In [ ]:
import pprint

pprint.pprint(sweep_config)

## 2. Initialize the Sweep

In [ ]:
# Initialize a new sweep
# Arguments:
#     – sweep_config: the sweep config dictionary defined above
#     – entity: Set the username for the sweep
#     – project: Set the project name for the sweep
sweep_id = wandb.sweep(sweep_config, entity="muse_ba", project="MUSE_drop_lsize_test")

### Define Your Neural Network
Before we can run the sweep, let's define a function that creates and trains our neural network.

In the function below, we define a simplified version of a VGG19 model in Keras, and add the following lines of code to log models metrics, visualize performance and output and track our experiments easily:
*   **wandb.init()** – Initialize a new W&B run. Each run is single execution of the training script.
*   **wandb.config** – Save all your hyperparameters in a config object. This lets you use our app to sort and compare your runs by hyperparameter values.
*   **callbacks=[WandbCallback()]** – Fetch all layer dimensions, model parameters and log them automatically to your W&B dashboard.
*   **wandb.log()** – Logs custom objects – these can be images, videos, audio files, HTML, plots, point clouds etc. Here we use wandb.log to log images of Simpson characters overlaid with actual and predicted labels.

In [ ]:
# The sweep calls this function with each set of hyperparameters
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 10,
        'batch_size': 128,
        'weight_decay': 0.0005,
        'learning_rate': 1e-3,
        'activation': 'relu',
        'optimizer': 'adam',
        'fc_layer_size1': 32,
        'fc_layer_size2': 64,
        'dropout': 0.2,
        'reccurent_dropout': 0.4,
        'kernel_regularizer': 0.01,
        'momentum': 0.9,
        'seed': 42,
        'sequence_length': 30,
        'beta_1': 0.9,
        'beta_2': 0.999,
        'clipnorm': 1.0
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    # Set random seed for reproducibility
    seed_value = 42
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

    # Set number of features and sequence lengths
    num_features = 16
    sequence_length = config.sequence_length

    columns_x = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                'ROCEIMPhase']

    columns_y = ['JointAngle', 'Mass']

    # Group the data by the 'Sample' column
    df_grouped = normalized_df.groupby(['Sample'])

    # Create sequences for each group
    X_seq, y_seq = [], []

    for group_name, group_data in df_grouped:

        group_data_x_temp = group_data[columns_x]
        group_data_x = np.array(group_data_x_temp)  # Convert to NumPy array

        group_data_temp_y = group_data[columns_y]
        group_data_y = np.array(group_data_temp_y)  # Convert to NumPy array

        # Create sequences
        for i in range(len(group_data) - sequence_length + 1):
            X_seq.append(group_data_x[i:i+sequence_length, :])
            y_seq.append(group_data_y[i+sequence_length-1, :])

    # Convert to numpy arrays
    X_seq_np = np.array(X_seq)
    y_seq_np = np.array(y_seq)

    # Split into training and validation data in a 80/20 ratio. Testing is done with the samples extracted from the dataset earlier
    X_train, X_val, y_train, y_val = train_test_split(X_seq_np, y_seq_np, test_size=0.2, random_state=42)


    # Build the GRU model
    model = Sequential()

    model.add(GRU(units=config.fc_layer_size1, activation='relu', return_sequences=True,
                  kernel_initializer=he_normal(seed=seed_value),
                  kernel_regularizer=l2(config.kernel_regularizer),
                  input_shape=(sequence_length, num_features), name='Input-Layer'))
    model.add(BatchNormalization())
    model.add(Dropout(config.dropout, seed=seed_value))  # Dropout after the first GRU layer

    model.add(GRU(units = config.fc_layer_size2, activation='relu',
                  kernel_initializer=he_normal(seed=seed_value),
                  kernel_regularizer=l2(config.kernel_regularizer),
                  recurrent_dropout=config.reccurent_dropout, dropout=config.dropout))
    model.add(BatchNormalization())
    model.add(Dropout(config.dropout, seed=seed_value))  # Dropout after the second GRU layer

    model.add(Dense(units=2, activation='linear'))  # Output layer

    # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = SGD(learning_rate=config.learning_rate, weight_decay=1e-5, momentum=config.momentum, nesterov=True)
    elif config.optimizer=='rmsprop':
      optimizer = RMSprop(learning_rate=config.learning_rate, weight_decay=1e-5)
    elif config.optimizer=='adam':
      optimizer = Adam(learning_rate=config.learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
    elif config.optimizer=='nadam':
      optimizer = Nadam(learning_rate=config.learning_rate, beta_1=config.beta_1, beta_2=config.beta_2, clipnorm=config.clipnorm)

    model.compile(loss = "mean_squared_error", optimizer = optimizer, metrics=['accuracy'])

    model.fit(X_train, y_train, batch_size=config.batch_size,
              epochs=config.epochs,
              validation_data=(X_val, y_val),
              callbacks=[WandbCallback(data_type="graph", validation_data=(X_val, y_val)),
                          EarlyStopping(patience=10, restore_best_weights=True)])

## 3. Run the sweep agent

In [ ]:
# Initialize a new sweep
# Arguments:
#     – sweep_id: the sweep_id to run - this was returned above by wandb.sweep()
#     – function: function that defines your model architecture and trains it
wandb.agent(sweep_id, train)

# Visualize Sweeps Results

## Parallel coordinates plot
This plot maps hyperparameter values to model metrics. It’s useful for honing in on combinations of hyperparameters that led to the best model performance.

![](https://assets.website-files.com/5ac6b7f2924c652fd013a891/5e190366778ad831455f9af2_s_194708415DEC35F74A7691FF6810D3B14703D1EFE1672ED29000BA98171242A5_1578695138341_image.png)

## Hyperparameter Importance Plot
The hyperparameter importance plot surfaces which hyperparameters were the best predictors of, and highly correlated to desirable values for your metrics.

![](https://assets.website-files.com/5ac6b7f2924c652fd013a891/5e190367778ad820b35f9af5_s_194708415DEC35F74A7691FF6810D3B14703D1EFE1672ED29000BA98171242A5_1578695757573_image.png)

These visualizations can help you save both time and resources running expensive hyperparameter optimizations by honing in on the parameters (and value ranges) that are the most important, and thereby worthy of further exploration.

# Next step - Get your hands dirty with sweeps
We created a simple training script and [a few flavors of sweep configs](https://github.com/wandb/examples/tree/master/keras-cnn-fashion) for you to play with. We highly encourage you to give these a try. This repo also has examples to help you try more advanced sweep features like [Bayesian Hyperband](https://app.wandb.ai/wandb/examples-keras-cnn-fashion/sweeps/us0ifmrf?workspace=user-lavanyashukla), and [Hyperopt](https://app.wandb.ai/wandb/examples-keras-cnn-fashion/sweeps/xbs2wm5e?workspace=user-lavanyashukla).

<h3>Further training</h3>

In [ ]:
loaded_checkpoint_path = 'final_model_Nadam_lz256x64_ES.h5'

# Load the model with custom_objects to recognize the GRU layer
loaded_model = load_model(loaded_checkpoint_path)

print("Model loaded successfully.")
print(loaded_model.summary())

In [ ]:
dir_all = 'all_sim_samples.csv'
df_all=pd.read_csv(dir_all, encoding='utf-8')
grouped = df_all.groupby('Sample')

# Set Pandas options to display more columns
pd.options.display.max_columns=150

# Calculate the median for each group
median_values = grouped[['EIMMagnitude', 'EIMPhase']].apply(lambda group: group[['EIMMagnitude', 'EIMPhase']].agg(['min', 'max']).median())

# Calculate the mean for each group
mean_values = grouped[['EIMMagnitude', 'EIMPhase']].mean()

# Calculate the standard deviation for each group
standard_deviations = grouped[['EIMMagnitude', 'EIMPhase']].std()

# Calculate the variance for each group
variance_values = grouped[['EIMMagnitude', 'EIMPhase']].var()

# Calculate the kurtosis for each group
kurtosis_values = grouped[['EIMMagnitude', 'EIMPhase']].apply(pd.DataFrame.kurtosis)

# Reset the index to get the 'Sample' column back
median_values.reset_index(inplace=True)
mean_values.reset_index(inplace=True)
standard_deviations.reset_index(inplace=True)
variance_values.reset_index(inplace=True)
kurtosis_values.reset_index(inplace=True)

# Rename the columns to indicate they represent the median
median_values.columns = ['Sample', 'MedianEIMMagnitude', 'MedianEIMPhase']
mean_values.columns = ['Sample', 'MeanEIMMagnitude', 'MeanEIMPhase']
standard_deviations.columns = ['Sample', 'StdEIMMagnitude', 'StdEIMPhase']
variance_values.columns = ['Sample', 'VarEIMMagnitude', 'VarEIMPhase']
kurtosis_values.columns = ['Sample', 'KurtEIMMagnitude', 'KurtEIMPhase']

# Merge the median and mean values back into the original DataFrame based on the 'Sample' column
df_all = df_all.merge(median_values, on='Sample', how='left')
df_all = df_all.merge(mean_values, on='Sample', how='left')
df_all = df_all.merge(standard_deviations, on='Sample', how='left')
df_all = df_all.merge(variance_values, on='Sample', how='left')
df_all = df_all.merge(kurtosis_values, on='Sample', how='left')

# Calculate rate of change for each group
df_all['ROCEIMMagnitude'] = df_all['RollingAverageMag'].pct_change()
df_all['ROCEIMPhase'] = df_all['RollingAveragePhase'].pct_change()

#Filling NaN values out with the first mean value in the series
ROC_value=df_all['ROCEIMMagnitude'][1]
df_all['ROCEIMMagnitude'].fillna(value=ROC_value, inplace=True)
ROC_value=df_all['ROCEIMPhase'][1]
df_all['ROCEIMPhase'].fillna(value=ROC_value, inplace=True)

In [ ]:
df_all

In [ ]:
# Display the original DataFrame length
print("Original DataFrame Length:", len(normalized_df))

df_all_grouped = df_all.groupby(['Sample'])

specific_samples = [3, 4, 7]

# Create an empty DataFrame to store the selected samples
df_test = pd.DataFrame()

# Iterate through the specific samples and extract them for testing.
for sample_value in specific_samples:
    if sample_value in df_all_grouped.groups:
        df_test = pd.concat([df_test, df_all_grouped.get_group(sample_value)])

# Save the test data to csv
df_test.to_csv('sim_test_samples.csv')

In [ ]:
df_test

In [ ]:
# Define the columns you want to normalize
features_to_normalize = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                        'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                        'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                        'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                        'ROCEIMPhase']

targets_to_normalize = ['Mass', 'JointAngle']

columns_to_use = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                        'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                        'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                        'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                        'ROCEIMPhase', 'Mass', 'JointAngle']

# Extract the 'Sample' column to append after normalization
sample_column = df_all['Sample'].values

# Create a copy of an original dataframe
df2=df_all.drop(['Time', 'Sample'], axis=1)

# Extracting mean and standard deviation for mean normalization
df_mean = df2.mean()
df_std = df2.std()

# Load the training datas mean and std
loaded_means = pd.read_csv('means.csv', index_col=0, squeeze=True)
loaded_std_devs = pd.read_csv('std_devs.csv', index_col=0, squeeze=True)

# Calculating the difference for proper scaling
delta_mean = df_mean - loaded_means
delta_std = df_std - loaded_std_devs

# Adjust the mean
adjusted_mean_new = df_mean - delta_mean
adjusted_std_new = df_std - delta_std

normalized_df=(df2-adjusted_mean_new)/adjusted_std_new

# Add the sample column again
normalized_df['Sample'] = sample_column

# Save means and std to CSV
# df_mean.to_csv('/content/drive/MyDrive/NeuralNetwork/means.csv', header=True)
# df_std.to_csv('/content/drive/MyDrive/NeuralNetwork/std_devs.csv', header=True)

# Show a snaphsot of data
normalized_df

In [ ]:
# Remove the extracted samples from the original DataFrame
normalized_df.drop(normalized_df[normalized_df['Sample'] == 3].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 4].index, inplace = True)
normalized_df.drop(normalized_df[normalized_df['Sample'] == 7].index, inplace = True)

# Display the modified original DataFrame length
print("\nModified Original DataFrame Length:", len(normalized_df))

# Check if test samples have been removed
for sample_value in specific_samples:
  print(sample_value in normalized_df['Sample'].unique())

In [ ]:
# Set number of features and sequence lengths
num_features = 16
sequence_length = 30

columns_x = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
            'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
            'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
            'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
            'ROCEIMPhase']

columns_y = ['JointAngle', 'Mass']

# Group the data by the 'Sample' column
df_grouped = normalized_df.groupby(['Sample'])

# Create sequences for each group
X_seq, y_seq = [], []

for group_name, group_data in df_grouped:

    group_data_x_temp = group_data[columns_x]
    group_data_x = np.array(group_data_x_temp)  # Convert to NumPy array

    group_data_temp_y = group_data[columns_y]
    group_data_y = np.array(group_data_temp_y)  # Convert to NumPy array

    # Create sequences
    for i in range(len(group_data) - sequence_length + 1):
        X_seq.append(group_data_x[i:i+sequence_length, :])
        y_seq.append(group_data_y[i+sequence_length-1, :])

# Convert to numpy arrays
X_seq_np = np.array(X_seq)
y_seq_np = np.array(y_seq)

# Split into training and validation data in a 80/20 ratio. Testing is done with the samples extracted from the dataset earlier
X_train, X_val, y_train, y_val = train_test_split(X_seq_np, y_seq_np, test_size=0.2, random_state=42)

In [ ]:
# Freeze the layers of the pre-trained model all except gru_1
for layer in model.layers:
    if layer.name == 'gru_1':
        layer.trainable = True
    else:
        layer.trainable = False

optimizer = Nadam(learning_rate = lr, beta_1 = 0.9, beta_2 = 0.999, clipvalue = 0.5)  # Clip gradients between -0.5 and 0.5 to address the exploding gradient problem
loaded_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
batch = 72
num_epochs = 200

# Train the model and record the history. Change model_checkpoint name as needed
checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Define the EarlyStopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',  # Choose the metric to monitor
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored metric
)

history = loaded_model.fit(X_train,
                    y_train,
                    epochs=num_epochs,
                    batch_size=72,
                    verbose=1,
                    callbacks=[model_checkpoint_callback, early_stopping_callback],
                    validation_data=(X_val, y_val),
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    validation_freq=1,
                   )

# Save the model. Adjust name accordingly
model_dir = 'final_model_Nadam_lz256x64_ES_NewSubj.h5'
loaded_model.save(model_dir)
print("Model saved successfully.")

<h1>Testing on trainings data subject<h1>

In [ ]:
loaded_checkpoint_path = 'final_model_Nadam_lz256x64_ES.h5'

# Load the model with custom_objects to recognize the GRU layer
loaded_model = load_model(loaded_checkpoint_path)

print("Model loaded successfully.")
print(loaded_model.summary())

In [ ]:
# Load the test set
dir_test = 'test_samples.csv'
df_test=pd.read_csv(dir_test, encoding='utf-8')

In [ ]:
df_test

In [ ]:
# Extract the 'Sample' column to append after normalization
sample_column = df_test['Sample'].values

# Create a copy of an original dataframe without Time and Sample
df_test=df_test.drop(['Time', 'Sample'], axis=1)

# Load means and stds back from CSV. Squeeze them into pandas series
loaded_means = pd.read_csv('adjusted_means.csv', index_col=0, squeeze=True)
loaded_std_devs = pd.read_csv('adjusted_std_devs.csv', index_col=0, squeeze=True)

# df_test_normalized=(df_test-df_mean)/df_std
df_test_normalized=(df_test-loaded_means)/loaded_std_devs

# Add the sample column again
df_test_normalized['Sample'] = sample_column

<h3>Sanity check if scale is applied correctly</h3>

In [ ]:
df_test

In [ ]:
df_test_normalized

In [ ]:
df_org = df_test_normalized*loaded_std_devs+loaded_means
df_org

In [ ]:
# Initialize test sequences
X_seq, y_seq = [], []
sequence_length = 30

df_grouped_test = df_test_normalized.groupby('Sample')

for group_name, group_data in df_grouped_test:

    group_data_temp = group_data[['EIMMagnitude', 'EIMPhase', 'RollingAverageMag',
                                  'RollingAveragePhase', 'MedianEIMMagnitude', 'MedianEIMPhase',
                                  'MeanEIMMagnitude', 'MeanEIMPhase', 'StdEIMMagnitude',
                                  'StdEIMPhase', 'VarEIMMagnitude', 'VarEIMPhase', 'KurtEIMMagnitude',
                                  'KurtEIMPhase', 'ROCEIMMagnitude','ROCEIMPhase']]

    group_data_x = np.array(group_data_temp)  # Convert to NumPy array

    group_data_temp = group_data[['JointAngle', 'Mass']]
    group_data_y = np.array(group_data_temp)  # Convert to NumPy array

    # Create sequences (adjust sequence_length as needed)
    for i in range(len(group_data) - sequence_length + 1):
        X_seq.append(group_data_x[i:i+sequence_length, :])
        y_seq.append(group_data_y[i+sequence_length-1, :])

# Convert to numpy arrays
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# Now you can use the new_model for predictions
predictions_scaled = loaded_model.predict(X_seq)

# Evaluate the model on the test set
loss, accuracy = loaded_model.evaluate(X_seq, y_seq)

In [ ]:
# Converting np.arrays to pd.dataframes to make rescaling easier
print(predictions_scaled.shape)
print(y_seq.shape)
predictions_df = pd.DataFrame(predictions_scaled, columns=['JointAngle', 'Mass'])
groundtruths_df = pd.DataFrame(y_seq, columns=['JointAngle', 'Mass'])
print(predictions_df.shape)
print(groundtruths_df.shape)

In [ ]:
# Isolate the needed means and standard deviations for reversing the scale
df_mean_pred = loaded_means[['JointAngle', 'Mass']]
df_std_pred = loaded_std_devs[['JointAngle', 'Mass']]

# Reversing the scale
predictions = predictions_df*df_std_pred+df_mean_pred
groundtruths = groundtruths_df*df_std_pred+df_mean_pred

In [ ]:
# Sorting into ground truths and predictions for plotting
joint_angle_true = groundtruths['JointAngle']
mass_true = groundtruths['Mass']

joint_angle_pred = predictions['JointAngle']
mass_pred = predictions['Mass']

# Plot the time series of Joint Angle
plt.figure(figsize=(12, 6))
plt.ylim([0, 200])
plt.plot(joint_angle_true, label='True Joint Angle', linewidth=6)
plt.plot(joint_angle_pred, label='Predicted Joint Angle')
plt.title('Joint Angle - True vs. Predicted')
plt.xlabel('Time')
plt.ylabel('Joint Angle')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.ylim([-2, 8])
plt.plot(mass_true, label='True Mass', linewidth=6)
plt.plot(mass_pred, label='Predicted Mass')
plt.title('Mass - True vs. Predicted')
plt.xlabel('Time')
plt.ylabel('Mass')
plt.legend()
plt.show()

<h1>Test on different subject</h1>

In [ ]:
dir_test = 'sim_test_samples.csv'
df_test_new=pd.read_csv(dir_test, encoding='utf-8')
grouped = df_test_new.groupby('Sample')

In [ ]:
# Set Pandas options to display more columns
pd.options.display.max_columns=150

# Calculate the median for each group
median_values = grouped[['EIMMagnitude', 'EIMPhase']].apply(lambda group: group[['EIMMagnitude', 'EIMPhase']].agg(['min', 'max']).median())

# Calculate the mean for each group
mean_values = grouped[['EIMMagnitude', 'EIMPhase']].mean()

# Calculate the standard deviation for each group
standard_deviations = grouped[['EIMMagnitude', 'EIMPhase']].std()

# Calculate the variance for each group
variance_values = grouped[['EIMMagnitude', 'EIMPhase']].var()

# Calculate the kurtosis for each group
kurtosis_values = grouped[['EIMMagnitude', 'EIMPhase']].apply(pd.DataFrame.kurtosis)

# Reset the index to get the 'Sample' column back
median_values.reset_index(inplace=True)
mean_values.reset_index(inplace=True)
standard_deviations.reset_index(inplace=True)
variance_values.reset_index(inplace=True)
kurtosis_values.reset_index(inplace=True)

# Rename the columns to indicate they represent the median
median_values.columns = ['Sample', 'MedianEIMMagnitude', 'MedianEIMPhase']
mean_values.columns = ['Sample', 'MeanEIMMagnitude', 'MeanEIMPhase']
standard_deviations.columns = ['Sample', 'StdEIMMagnitude', 'StdEIMPhase']
variance_values.columns = ['Sample', 'VarEIMMagnitude', 'VarEIMPhase']
kurtosis_values.columns = ['Sample', 'KurtEIMMagnitude', 'KurtEIMPhase']

# Merge the median and mean values back into the original DataFrame based on the 'Sample' column
df_test_new = df_test_new.merge(median_values, on='Sample', how='left')
df_test_new = df_test_new.merge(mean_values, on='Sample', how='left')
df_test_new = df_test_new.merge(standard_deviations, on='Sample', how='left')
df_test_new = df_test_new.merge(variance_values, on='Sample', how='left')
df_test_new = df_test_new.merge(kurtosis_values, on='Sample', how='left')

# Calculate rate of change for each group
df_test_new['ROCEIMMagnitude'] = df_test_new['RollingAverageMag'].pct_change()
df_test_new['ROCEIMPhase'] = df_test_new['RollingAveragePhase'].pct_change()

#Filling NaN values out with the first mean value in the series
ROC_value=df_test_new['ROCEIMMagnitude'][1]
df_test_new['ROCEIMMagnitude'].fillna(value=ROC_value, inplace=True)
ROC_value=df_test_new['ROCEIMPhase'][1]
df_test_new['ROCEIMPhase'].fillna(value=ROC_value, inplace=True)

In [ ]:
df_test_new

In [ ]:
# Define the columns you want to normalize
features_to_normalize = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                        'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                        'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                        'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                        'ROCEIMPhase']

targets_to_normalize = ['Mass', 'JointAngle']

columns_to_use = ['EIMMagnitude', 'EIMPhase', 'RollingAverageMag', 'RollingAveragePhase',
                        'MedianEIMMagnitude', 'MedianEIMPhase', 'MeanEIMMagnitude',
                        'MeanEIMPhase', 'StdEIMMagnitude', 'StdEIMPhase', 'VarEIMMagnitude',
                        'VarEIMPhase', 'KurtEIMMagnitude', 'KurtEIMPhase', 'ROCEIMMagnitude',
                        'ROCEIMPhase', 'Mass', 'JointAngle']

# Extract the 'Sample' column to append after normalization
sample_column = df_test_new['Sample'].values

# Create a copy of an original dataframe
df2=df_test_new.drop(['Time', 'Sample'], axis=1)

# Extracting mean and standard deviation for mean normalization
df_mean = df2.mean()
df_std = df2.std()

# Load the training datas mean and std
loaded_means = pd.read_csv('means.csv', index_col=0, squeeze=True)
loaded_std_devs = pd.read_csv('std_devs.csv', index_col=0, squeeze=True)

# Calculating the difference for proper scaling
delta_mean = df_mean - loaded_means
delta_std = df_std - loaded_std_devs

# Adjust the mean
adjusted_mean_new = df_mean - delta_mean
adjusted_std_new = df_std - delta_std

normalized_df=(df2-adjusted_mean_new)/adjusted_std_new

# Add the sample column again
normalized_df['Sample'] = sample_column

# Save means and std to CSV
# adjusted_mean_new.to_csv('adjusted_means.csv', header=True)
# adjusted_std_new.to_csv('adjusted_std_devs.csv', header=True)

# Show a snaphsot of data
normalized_df

In [ ]:
df_org = normalized_df*adjusted_std_new+adjusted_mean_new
df_org

In [ ]:
# Initialize test sequences
X_seq, y_seq = [], []
sequence_length = 30

df_grouped_test = normalized_df.groupby('Sample')

for group_name, group_data in df_grouped_test:

    group_data_temp = group_data[['EIMMagnitude', 'EIMPhase', 'RollingAverageMag',
                                  'RollingAveragePhase', 'MedianEIMMagnitude', 'MedianEIMPhase',
                                  'MeanEIMMagnitude', 'MeanEIMPhase', 'StdEIMMagnitude',
                                  'StdEIMPhase', 'VarEIMMagnitude', 'VarEIMPhase', 'KurtEIMMagnitude',
                                  'KurtEIMPhase', 'ROCEIMMagnitude','ROCEIMPhase']]

    group_data_x = np.array(group_data_temp)  # Convert to NumPy array

    group_data_temp = group_data[['JointAngle', 'Mass']]
    group_data_y = np.array(group_data_temp)  # Convert to NumPy array

    # Create sequences (adjust sequence_length as needed)
    for i in range(len(group_data) - sequence_length + 1):
        X_seq.append(group_data_x[i:i+sequence_length, :])
        y_seq.append(group_data_y[i+sequence_length-1, :])

# Convert to numpy arrays
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# Now you can use the new_model for predictions
predictions_scaled = loaded_model.predict(X_seq)

# Evaluate the model on the test set
loss, accuracy = loaded_model.evaluate(X_seq, y_seq)

In [ ]:
# Converting np.arrays to pd.dataframes to make rescaling easier
print(predictions_scaled.shape)
print(y_seq.shape)
predictions_df = pd.DataFrame(predictions_scaled, columns=['JointAngle', 'Mass'])
groundtruths_df = pd.DataFrame(y_seq, columns=['JointAngle', 'Mass'])
print(predictions_df.shape)
print(groundtruths_df.shape)

In [ ]:
# Isolate the needed means and standard deviations for reversing the scale
df_mean_pred = adjusted_mean_new[['JointAngle', 'Mass']]
df_std_pred = adjusted_std_new[['JointAngle', 'Mass']]

# Reversing the scale
predictions = predictions_df*df_std_pred+df_mean_pred
groundtruths = groundtruths_df*df_std_pred+df_mean_pred

In [ ]:
# Sorting into ground truths and predictions for plotting
joint_angle_true = groundtruths['JointAngle']
mass_true = groundtruths['Mass']

joint_angle_pred = predictions['JointAngle']
mass_pred = predictions['Mass']

# Plot the time series of Joint Angle
plt.figure(figsize=(12, 6))
plt.ylim([0, 200])
plt.plot(joint_angle_true, label='True Joint Angle', linewidth=6)
plt.plot(joint_angle_pred, label='Predicted Joint Angle')
plt.title('Joint Angle - True vs. Predicted')
plt.xlabel('Time')
plt.ylabel('Joint Angle')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.ylim([-2, 8])
plt.plot(mass_true, label='True Mass', linewidth=6)
plt.plot(mass_pred, label='Predicted Mass')
plt.title('Mass - True vs. Predicted')
plt.xlabel('Time')
plt.ylabel('Mass')
plt.legend()
plt.show()